In [ ]:
import numpy as np
import os
import sys

import wave
import copy
import math

from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten, Merge, Embedding, Convolution1D,Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import Conv2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import label_binarize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence


from features import *
from helper import *

Using TensorFlow backend.


In [ ]:
code_path = os.path.dirname(os.path.realpath(os.getcwd()))
emotions_used = np.array(['ang', 'exc', 'neu', 'sad','frus','surp'])
data_path = code_path + "/../data/sessions/"
sessions = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
framerate = 16000

In [ ]:
import pickle
with open(data_path + '/../'+'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [ ]:
text = []

for ses_mod in data2:
    text.append(ses_mod['transcription'])
    
MAX_SEQUENCE_LENGTH = 500

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
x_train_text = []

x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
x_train_speech = []

counter = 0
for ses_mod in data2:
    x_head = ses_mod['signal']
    st_features = calculate_features(x_head, framerate, None)
    st_features, _ = pad_sequence_into_array(st_features, maxlen=100)
    x_train_speech.append( st_features.T )
    counter+=1
    if(counter%100==0):
        print(counter)
    
x_train_speech = np.array(x_train_speech)
x_train_speech.shape

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


(4936, 100, 34)

In [ ]:
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(4936, 200, 189, 1)

In [ ]:
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = label_binarize(Y,emotions_used)

Y.shape

(4936, 4)

 # TEXT-SPEECH MODEL 1

In [ ]:
model_text = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model_text.add(Embedding(2736,
                    128,input_length=500))
model_text.add(Flatten())
model_text.add(Dense(1024))
model_text.add(Activation('relu'))
model_text.add(Dropout(0.2))
model_text.add(Dense(512))
model_text.add(Activation('relu'))
model_text.add(Dropout(0.2))
model_text.add(Dense(256))



model_speech = Sequential()
model_speech.add(Flatten(input_shape=(100, 34)))
model_speech.add(Dense(1024))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.2))
model_speech.add(Dense(512))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.2))
model_speech.add(Dense(256))


model_combined = Sequential()
model_combined.add(Merge([model_text, model_speech], mode='concat'))

model_combined.add(Activation('relu'))
model_combined.add(Dense(256))
model_combined.add(Activation('relu'))

model_combined.add(Dense(6))
model_combined.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_combined.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['acc'])


model_speech.summary()
model_text.summary()
model_combined.summary()

print("Model1 Built")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_21 (Flatten)         (None, 3400)              0         
_________________________________________________________________
dense_59 (Dense)             (None, 1024)              3482624   
_________________________________________________________________
activation_65 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_49 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_60 (Dense)             (None, 512)               524800    
_________________________________________________________________
activation_66 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_50 (Dropout)         (None, 512)               0         
__________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:28: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [ ]:
hist = model_combined.fit([x_train_text,x_train_speech], Y, 
                 batch_size=64, nb_epoch=30, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/30
3948/3948 [==============================] - 7s - loss: 2.3053 - acc: 0.3214 - val_loss: 1.3071 - val_acc: 0.4322
Epoch 2/30
3948/3948 [==============================] - 6s - loss: 1.1983 - acc: 0.4415 - val_loss: 1.2201 - val_acc: 0.4858
Epoch 3/30
3948/3948 [==============================] - 6s - loss: 1.1503 - acc: 0.4549 - val_loss: 1.1888 - val_acc: 0.4777
Epoch 4/30
3948/3948 [==============================] - 6s - loss: 0.9828 - acc: 0.5595 - val_loss: 1.0691 - val_acc: 0.6063
Epoch 5/30
3948/3948 [==============================] - 6s - loss: 0.5954 - acc: 0.7617 - val_loss: 0.9443 - val_acc: 0.6518
Epoch 6/30
3948/3948 [==============================] - 6s - loss: 0.3732 - acc: 0.8609 - val_loss: 1.1165 - val_acc: 0.6387
Epoch 7/30
3948/3948 [==============================] - 6s - loss: 0.2703 - acc: 0.9017 - val_loss: 1.3433 - val_acc: 0.6488
Epoch 8/30
3948/3948 [==============================] - 6s - loss: 0.2165 - ac

#**TEXT - SPEECH MODEL 2**

In [ ]:
model_text = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model_text.add(Embedding(2736,
                    128,input_length=500))

model_text.add(LSTM(256, return_sequences=True, input_shape=(100, 34)))
model_text.add(LSTM(256, return_sequences=False))
model_text.add(Dense(256))


model_speech = Sequential()
model_speech.add(Flatten(input_shape=(100, 34)))
model_speech.add(Dense(1024))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.2))
model_speech.add(Dense(256))


model_combined = Sequential()
model_combined.add(Merge([model_text, model_speech], mode='concat'))

model_combined.add(Activation('relu'))
model_combined.add(Dense(256))
model_combined.add(Activation('relu'))

model_combined.add(Dense(6))
model_combined.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_combined.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['acc'])


model_speech.summary()
model_text.summary()
model_combined.summary()

print("Model2 Built")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_23 (Flatten)         (None, 3400)              0         
_________________________________________________________________
dense_70 (Dense)             (None, 1024)              3482624   
_________________________________________________________________
activation_73 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_52 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_71 (Dense)             (None, 256)               262400    
Total params: 3,745,024
Trainable params: 3,745,024
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [ ]:
hist = model_combined.fit([x_train_text,x_train_speech], Y, 
                 batch_size=64, nb_epoch=10, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/10
3948/3948 [==============================] - 96s - loss: 1.5177 - acc: 0.3883 - val_loss: 1.1521 - val_acc: 0.5030
Epoch 2/10
3948/3948 [==============================] - 95s - loss: 1.0155 - acc: 0.5585 - val_loss: 1.0288 - val_acc: 0.5911
Epoch 3/10
3948/3948 [==============================] - 95s - loss: 0.8338 - acc: 0.6537 - val_loss: 1.1359 - val_acc: 0.5911
Epoch 4/10
3948/3948 [==============================] - 96s - loss: 0.6285 - acc: 0.7457 - val_loss: 1.0116 - val_acc: 0.6569
Epoch 5/10
3948/3948 [==============================] - 95s - loss: 0.5209 - acc: 0.8045 - val_loss: 0.9996 - val_acc: 0.6538
Epoch 6/10
3948/3948 [==============================] - 96s - loss: 0.4503 - acc: 0.8247 - val_loss: 1.0920 - val_acc: 0.6528
Epoch 7/10
3948/3948 [==============================] - 95s - loss: 0.3838 - acc: 0.8531 - val_loss: 1.0566 - val_acc: 0.6630
Epoch 8/10
3948/3948 [==============================] - 95s - loss: 0.3

#TEXT - SPEECH MODEL 3

In [ ]:
model_text = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model_text.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))

model_text.add(LSTM(256, return_sequences=True))
model_text.add(LSTM(256, return_sequences=False))
model_text.add(Dense(256))


model_speech = Sequential()
model_speech.add(Flatten(input_shape=(100, 34)))
model_speech.add(Dense(1024))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.2))
model_speech.add(Dense(256))


model_combined = Sequential()
model_combined.add(Merge([model_text, model_speech], mode='concat'))

model_combined.add(Activation('relu'))
model_combined.add(Dense(256))
model_combined.add(Activation('relu'))

model_combined.add(Dense(4))
model_combined.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_combined.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['acc'])


model_speech.summary()
model_text.summary()
model_combined.summary()

print("Model3 Built")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_24 (Flatten)         (None, 3400)              0         
_________________________________________________________________
dense_75 (Dense)             (None, 1024)              3482624   
_________________________________________________________________
activation_77 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_53 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_76 (Dense)             (None, 256)               262400    
Total params: 3,745,024
Trainable params: 3,745,024
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:23: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [ ]:
hist = model_combined.fit([x_train_text,x_train_speech], Y, 
                 batch_size=64, nb_epoch=10, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/10
3948/3948 [==============================] - 99s - loss: 1.4061 - acc: 0.3982 - val_loss: 1.1408 - val_acc: 0.5192
Epoch 2/10
3948/3948 [==============================] - 98s - loss: 0.9457 - acc: 0.6013 - val_loss: 1.0741 - val_acc: 0.5648
Epoch 3/10
3948/3948 [==============================] - 96s - loss: 0.7025 - acc: 0.7171 - val_loss: 0.8983 - val_acc: 0.6559
Epoch 4/10
3948/3948 [==============================] - 97s - loss: 0.5498 - acc: 0.7781 - val_loss: 0.8956 - val_acc: 0.6913
Epoch 5/10
3948/3948 [==============================] - 97s - loss: 0.4412 - acc: 0.8311 - val_loss: 0.9891 - val_acc: 0.6761
Epoch 6/10
3948/3948 [==============================] - 97s - loss: 0.3530 - acc: 0.8642 - val_loss: 0.9266 - val_acc: 0.6751
Epoch 7/10
3948/3948 [==============================] - 97s - loss: 0.3004 - acc: 0.8891 - val_loss: 1.0780 - val_acc: 0.6832
Epoch 8/10
3948/3948 [==============================] - 97s - loss: 0.2

#**TRI MODEL 1**

In [ ]:
model_text = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model_text.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))

model_text.add(LSTM(256, return_sequences=True))
model_text.add(LSTM(256, return_sequences=False))
model_text.add(Dense(256))


model_speech = Sequential()
model_speech.add(Flatten(input_shape=(100, 34)))
model_speech.add(Dense(1024))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.2))
model_speech.add(Dense(256))

model_mocap = Sequential()
model_mocap.add(Conv2D(32, 3, strides=(2, 2), border_mode='same', input_shape=(200, 189, 1)))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Flatten())
model_mocap.add(Dense(256))

model_combined = Sequential()
model_combined.add(Merge([model_text, model_speech, model_mocap], mode='concat'))

model_combined.add(Activation('relu'))

model_combined.add(Dense(256))
model_combined.add(Activation('relu'))

model_combined.add(Dense(6))
model_combined.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_combined.compile(loss='categorical_crossentropy',optimizer='Adam' ,metrics=['acc'])


model_speech.summary()
model_text.summary()
model_mocap.summary()
model_combined.summary()

print("Model1 Built")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_29 (Flatten)         (None, 3400)              0         
_________________________________________________________________
dense_93 (Dense)             (None, 1024)              3482624   
_________________________________________________________________
activation_100 (Activation)  (None, 1024)              0         
_________________________________________________________________
dropout_67 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_94 (Dense)             (None, 256)               262400    
Total params: 3,745,024
Trainable params: 3,745,024
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, strides=(2, 2), padding="same", input_shape=(200, 189,...)`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:41: UserWarning: The `Merge` layer is deprecated a

In [ ]:
hist = model_combined.fit([x_train_text,x_train_speech,x_train_mocap], Y, 
                 batch_size=64, nb_epoch=20, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/20
3948/3948 [==============================] - 104s - loss: 2.1268 - acc: 0.3734 - val_loss: 1.1610 - val_acc: 0.4949
Epoch 2/20
3948/3948 [==============================] - 100s - loss: 0.9963 - acc: 0.5732 - val_loss: 0.9263 - val_acc: 0.6326
Epoch 3/20
3948/3948 [==============================] - 100s - loss: 0.7343 - acc: 0.7085 - val_loss: 0.8728 - val_acc: 0.6650
Epoch 4/20
3948/3948 [==============================] - 101s - loss: 0.5629 - acc: 0.7804 - val_loss: 0.8316 - val_acc: 0.6781
Epoch 5/20
3948/3948 [==============================] - 100s - loss: 0.4776 - acc: 0.8179 - val_loss: 0.9013 - val_acc: 0.6802
Epoch 6/20
3948/3948 [==============================] - 101s - loss: 0.3749 - acc: 0.8597 - val_loss: 0.9670 - val_acc: 0.6832
Epoch 7/20
3948/3948 [==============================] - 100s - loss: 0.3061 - acc: 0.8827 - val_loss: 0.9723 - val_acc: 0.6852
Epoch 8/20
3948/3948 [==============================] - 100s - l

#TRI MODEL 2

In [ ]:
model_text = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model_text.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))
model_text.add(Convolution1D(256, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Convolution1D(128, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Convolution1D(64, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Convolution1D(32, 3, border_mode='same'))
model_text.add(Dropout(0.2))
model_text.add(Activation('relu'))
model_text.add(Flatten())
model_text.add(Dropout(0.2))
model_text.add(Dense(256))


model_speech = Sequential()
model_speech.add(Flatten(input_shape=(100, 34)))
model_speech.add(Dense(1024))
model_speech.add(Activation('relu'))
model_speech.add(Dropout(0.2))
model_speech.add(Dense(256))

model_mocap = Sequential()
model_mocap.add(Conv2D(32, 3, strides=(2, 2), border_mode='same', input_shape=(200, 189, 1)))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Flatten())
model_mocap.add(Dense(256))

model_combined = Sequential()
model_combined.add(Merge([model_text, model_speech, model_mocap], mode='concat'))
model_combined.add(Activation('relu'))

model_combined.add(Dense(256))
model_combined.add(Activation('relu'))

model_combined.add(Dense(6))
model_combined.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_combined.compile(loss='categorical_crossentropy',optimizer='Adam' ,metrics=['acc'])


model_speech.summary()
model_text.summary()
model_mocap.summary()
model_combined.summary()

print("Model2 Built")


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, 3, padding="same")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, strides=(2, 2), padding="same", input_shape=(200, 189,...)`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the K

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_35 (Flatten)         (None, 3400)              0         
_________________________________________________________________
dense_106 (Dense)            (None, 1024)              3482624   
_________________________________________________________________
activation_126 (Activation)  (None, 1024)              0         
_________________________________________________________________
dropout_90 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_107 (Dense)            (None, 256)               262400    
Total params: 3,745,024
Trainable params: 3,745,024
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:42: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:52: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [ ]:
hist = model_combined.fit([x_train_text,x_train_speech,x_train_mocap], Y, 
                 batch_size=64, nb_epoch=60, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/60
3948/3948 [==============================] - 14s - loss: 2.1924 - acc: 0.3336 - val_loss: 1.2536 - val_acc: 0.4332
Epoch 2/60
3948/3948 [==============================] - 13s - loss: 1.2030 - acc: 0.4397 - val_loss: 1.1904 - val_acc: 0.4626
Epoch 3/60
3948/3948 [==============================] - 13s - loss: 1.1540 - acc: 0.4514 - val_loss: 1.1939 - val_acc: 0.4656
Epoch 4/60
3948/3948 [==============================] - 13s - loss: 1.1148 - acc: 0.4792 - val_loss: 1.2296 - val_acc: 0.4555
Epoch 5/60
3948/3948 [==============================] - 13s - loss: 1.0330 - acc: 0.5433 - val_loss: 1.0895 - val_acc: 0.5536A: 3s - loss: 
Epoch 6/60
3948/3948 [==============================] - 13s - loss: 0.9839 - acc: 0.5704 - val_loss: 1.1032 - val_acc: 0.5617
Epoch 7/60
3948/3948 [==============================] - 13s - loss: 0.9574 - acc: 0.5912 - val_loss: 1.0948 - val_acc: 0.5283
Epoch 8/60
3948/3948 [==============================] - 1

#TRI MODEL 3

In [ ]:
model_text = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model_text.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))

model_text.add(LSTM(256, return_sequences=True))
model_text.add(LSTM(256, return_sequences=False))
model_text.add(Dense(256))


model_speech = Sequential()
model_speech.add(LSTM(128, return_sequences=True, input_shape=(100, 34)))
model_speech.add(AttentionDecoder(128,128))
model_speech.add(Flatten())
model_speech.add(Dense(256))

model_mocap = Sequential()
model_mocap.add(Conv2D(32, 3, strides=(2, 2), border_mode='same', input_shape=(200, 189, 1)))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Flatten())
model_mocap.add(Dense(256))

model_combined = Sequential()
model_combined.add(Merge([model_text, model_speech, model_mocap], mode='concat'))

model_combined.add(Activation('relu'))

model_combined.add(Dense(256))
model_combined.add(Activation('relu'))

model_combined.add(Dense(6))
model_combined.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_combined.compile(loss='categorical_crossentropy',optimizer='Adam' ,metrics=['acc'])


model_speech.summary()
model_text.summary()
model_mocap.summary()
model_combined.summary()

print("Model3 Built")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_23 (LSTM)               (None, 100, 128)          83456     
_________________________________________________________________
AttentionDecoder (AttentionD (None, 100, 128)          246528    
_________________________________________________________________
flatten_43 (Flatten)         (None, 12800)             0         
_________________________________________________________________
dense_120 (Dense)            (None, 256)               3277056   
Total params: 3,607,040
Trainable params: 3,607,040
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (None, 500, 300)          821100    
_________________________________________________________________


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, strides=(2, 2), padding="same", input_shape=(200, 189,...)`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, strides=(2, 2), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:40: UserWarning: The `Merge` layer is deprecated a

In [ ]:
hist = model_combined.fit([x_train_text,x_train_speech,x_train_mocap], Y, 
                 batch_size=64, nb_epoch=30, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/30
3948/3948 [==============================] - 134s - loss: 1.4364 - acc: 0.3652 - val_loss: 1.2002 - val_acc: 0.5040
Epoch 2/30
3948/3948 [==============================] - 131s - loss: 1.0254 - acc: 0.5795 - val_loss: 1.0986 - val_acc: 0.5567
Epoch 3/30
3948/3948 [==============================] - 131s - loss: 0.7341 - acc: 0.7054 - val_loss: 0.9238 - val_acc: 0.6569
Epoch 4/30
3948/3948 [==============================] - 132s - loss: 0.5781 - acc: 0.7781 - val_loss: 0.9490 - val_acc: 0.6377
Epoch 5/30
3948/3948 [==============================] - 131s - loss: 0.4860 - acc: 0.8078 - val_loss: 0.8963 - val_acc: 0.6731
Epoch 6/30
3948/3948 [==============================] - 131s - loss: 0.3813 - acc: 0.8549 - val_loss: 0.9789 - val_acc: 0.6781
Epoch 7/30
3948/3948 [==============================] - 132s - loss: 0.3127 - acc: 0.8756 - val_loss: 1.0113 - val_acc: 0.6640
Epoch 8/30
3948/3948 [==============================] - 132s - l